In [ ]:
import shutil

shutil.rmtree('/content/original_data_set/freshoranges',ignore_errors= True)

In [ ]:
shutil.rmtree('/content/original_data_set/original_data_set',ignore_errors= True)

In [ ]:
shutil.rmtree('/content/original_data_set/rottenoranges',ignore_errors= True)

In [ ]:
!mv /content/original_data_set/rottenapples/*.jpg /content/original_data_set/freshapples

In [ ]:
!cp -r /content/original_data_set /content/drive/MyDrive/apple_banana

In [ ]:
import os
import shutil

# Define the directories containing the original images
class1_dir = '/content/original_data_set/freshapples'
class2_dir = '/content/original_data_set/freshbanana'

# Define the base directory where the train and validation folders will be created
base_dir = '/content/drive/MyDrive/apple_banana/data'

# Define the names of the train and validation folders
train_dir = 'train'
val_dir = 'test'

# Define the names of the class folders within the train and validation folders
class1_train_dir = 'apple'
class2_train_dir = 'banana'
class1_val_dir = 'apple'
class2_val_dir = 'banana'

# Define the paths to the train and validation folders
train_path = os.path.join(base_dir, train_dir)
val_path = os.path.join(base_dir, val_dir)

# Create the train and validation folders if they don't already exist
if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(val_path):
    os.makedirs(val_path)

# Create the class1 and class2 folders within the train and validation folders
class1_train_path = os.path.join(train_path, class1_train_dir)
class2_train_path = os.path.join(train_path, class2_train_dir)
class1_val_path = os.path.join(val_path, class1_val_dir)
class2_val_path = os.path.join(val_path, class2_val_dir)

if not os.path.exists(class1_train_path):
    os.makedirs(class1_train_path)
if not os.path.exists(class2_train_path):
    os.makedirs(class2_train_path)
if not os.path.exists(class1_val_path):
    os.makedirs(class1_val_path)
if not os.path.exists(class2_val_path):
    os.makedirs(class2_val_path)

# Move images from class1 directory into the train and validation folders
class1_imgs = os.listdir(class1_dir)
n_class1 = len(class1_imgs)
n_train_class1 = int(n_class1 * 0.8) # 80% for training
n_val_class1 = n_class1 - n_train_class1 # remaining 20% for validation

for i, img_name in enumerate(class1_imgs):
    img_path = os.path.join(class1_dir, img_name)
    if i < n_train_class1:
        # move image to training folder
        shutil.copy(img_path, os.path.join(class1_train_path, img_name))
    else:
        # move image to validation folder
        shutil.copy(img_path, os.path.join(class1_val_path, img_name))

# Move images from class2 directory into the train and validation folders
class2_imgs = os.listdir(class2_dir)
n_class2 = len(class2_imgs)
n_train_class2 = int(n_class2 * 0.8) # 80% for training
n_val_class2 = n_class2 - n_train_class2 # remaining 20% for validation

for i, img_name in enumerate(class2_imgs):
    img_path = os.path.join(class2_dir, img_name)
    if i < n_train_class2:
        # move image to training folder
        shutil.copy(img_path, os.path.join(class2_train_path, img_name))
    else:
        # move image to validation folder
        shutil.copy(img_path, os.path.join(class2_val_path, img_name))


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define input shape and number of classes
img_shape = (224, 224, 3)
num_classes = 2

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/apple_banana/data/train',
    target_size=img_shape[:2],
    batch_size=32,
    class_mode='categorical'   # change class_mode to 'categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/apple_banana/data/test',
    target_size=img_shape[:2],
    batch_size=32,
    class_mode='categorical'   # change class_mode to 'categorical'
)

Found 360 images belonging to 2 classes.
Found 91 images belonging to 2 classes.


In [ ]:
# Define function to preprocess images to HSV and YCbCr
def preprocess(img):
    hsv = tf.image.rgb_to_hsv(img)
    ycbcr = tf.image.rgb_to_yuv(img)
    return tf.concat([img, hsv, ycbcr], axis=-1)


In [ ]:
# Define MobileNet with modified first convolutional layer, preprocessing, and global average pooling
inputs = Input(shape=img_shape)
x = Lambda(preprocess)(inputs)

x = Conv2D(3, (3, 3), strides=(2, 2), padding='same', name='conv1')(x)



x = Activation('relu')(x)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
base_model = tf.keras.applications.MobileNet(
    input_shape=(112, 112, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

x = base_model(x)
outputs = Dense(num_classes, activation='sigmoid')(x)
# x = Dense(256, activation='relu')(x)
# outputs = Dense(128, activation='relu')(x)

model = Model(inputs, outputs)

x ==========>>>>>>>> KerasTensor(type_spec=TensorSpec(shape=(None, 112, 112, 3), dtype=tf.float32, name=None), name='tf.math.subtract/Sub:0', description="created by layer 'tf.math.subtract'")


In [ ]:
for layer in base_model.layers[:-10]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=1000,
    validation_data=test_generator
)

Epoch 1/1000
12/12 [==============================] - 12s 690ms/step - loss: 0.0648 - accuracy: 0.9778 - val_loss: 1.7364 - val_accuracy: 0.5934
Epoch 2/1000
12/12 [==============================] - 6s 521ms/step - loss: 0.0506 - accuracy: 0.9861 - val_loss: 0.2068 - val_accuracy: 0.9231
Epoch 3/1000
12/12 [==============================] - 8s 681ms/step - loss: 0.0846 - accuracy: 0.9722 - val_loss: 0.0268 - val_accuracy: 1.0000
Epoch 4/1000
12/12 [==============================] - 6s 534ms/step - loss: 0.0740 - accuracy: 0.9694 - val_loss: 2.7541 - val_accuracy: 0.5604
Epoch 5/1000
12/12 [==============================] - 7s 626ms/step - loss: 0.0821 - accuracy: 0.9694 - val_loss: 4.7948 - val_accuracy: 0.5165
Epoch 6/1000
12/12 [==============================] - 6s 525ms/step - loss: 0.0925 - accuracy: 0.9639 - val_loss: 1.1969 - val_accuracy: 0.6264
Epoch 7/1000
12/12 [==============================] - 8s 641ms/step - loss: 0.1585 - accuracy: 0.9528 - val_loss: 1.1115 - val_accuracy

In [ ]:
# Evaluate model on test set
loss, accuracy = model.evaluate(test_generator)

# Print test set accuracy
print("Test set accuracy: {:.2f}%".format(accuracy * 100))


3/3 [==============================] - 1s 175ms/step - loss: 0.0017 - accuracy: 1.0000
Test set accuracy: 100.00%


In [ ]:
# Generate predictions for test set
predictions = model.predict(test_generator)

# Print predicted class probabilities for the first image in the test set
print("Predicted class probabilities:", predictions[0])


3/3 [==============================] - 1s 181ms/step
Predicted class probabilities: [1.0000000e+00 3.0168028e-13]
3/3 [==============================] - 1s 168ms/step
Predicted class probabilities: [1.2064759e-06 9.9999809e-01]


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the saved model
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/apple_vs_banana.h5')

# Load the new image
img = load_img('/content/banana.jpeg', target_size=img_shape)

# Preprocess the image
img_array = img_to_array(img)
img_array = img_array[np.newaxis, ...] / 255.

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the predicted class label
class_labels = train_generator.class_indices
predicted_label = np.argmax(predictions)
predicted_class = list(class_labels.keys())[list(class_labels.values()).index(predicted_label)]

# Print the predicted class
print("Predicted class:", predicted_class)

1/1 [==============================] - 1s 505ms/step
Predicted class: banana


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Define input shape and number of classes
img_shape = (224, 224, 3)
num_classes = 2

# Load the saved model
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/apple_vs_banana.h5')

# Define data generator for test set
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/apple_banana/data/test',
    target_size=img_shape[:2],
    batch_size=1,
    shuffle=False,
    class_mode='categorical'
)

# Generate predictions for test set
predictions = loaded_model.predict(test_generator)

# Get true labels for test set
true_labels = test_generator.classes

# Convert predicted labels to class indices
predicted_labels = np.argmax(predictions, axis=-1)

# Generate confusion matrix and classification report
confusion = confusion_matrix(true_labels, predicted_labels)
report = classification_report(true_labels, predicted_labels, target_names=list(test_generator.class_indices.keys()))

# Print confusion matrix and classification report
print("Confusion Matrix:\n", confusion)
print("\nClassification Report:\n", report)


Found 91 images belonging to 2 classes.


91/91 [==============================] - 41s 446ms/step
Confusion Matrix:
 [[47  0]
 [ 0 44]]

Classification Report:
               precision    recall  f1-score   support

       apple       1.00      1.00      1.00        47
      banana       1.00      1.00      1.00        44

    accuracy                           1.00        91
   macro avg       1.00      1.00      1.00        91
weighted avg       1.00      1.00      1.00        91

